In [ ]:
from NeuralNetwork import NeuralNetwork, train_nn, performance, split
from _functions import *
import numpy as np
from time import time

start = time()

# Get a list of all the client's models
client_models = get_dirlist("models")
client_models.remove("server")

# Find the "servers" weights
# This, by default is the first trained model that a client spits out
serverWIH= np.load("models/" + client_models[0] +"/wih.npy")
serverWHO = np.load("models/" + client_models[0]+ "/who.npy")

totalAcc = 0

# Combine/Merge all the models
for model in client_models:
    
    clientWIH = np.load("models/" + model +"/wih.npy")
    clientWHO = np.load("models/" + model +"/who.npy")
    
    serverWIH = np.mean( np.array([ serverWIH, clientWIH ]), axis=0 )
    serverWHO = np.mean( np.array([ serverWHO, clientWHO ]), axis=0 )
    
    # number of input, hidden and output nodes
    input_nodes = 784
    hidden_nodes = 200
    output_nodes = 10
    learning_rate = 0.1

    # Create the neural network object
    n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
    n.update_info(serverWIH, serverWHO)
    
    perf = performance(n)
    totalAcc += perf
    print("Server performance after merge with " + model +": " + str(perf))

print("Final server performance: " + str(round(totalAcc / len(client_models), ndigits=3)))
print("")
print("Merging took: " + str(round(time() - start, ndigits = 3)) + " seconds")

Server performance after merge with mnist1525444510.9865973: 0.9401
Server performance after merge with mnist1525444567.1967733: 0.9416
Server performance after merge with mnist1525444626.6895838: 0.945
Server performance after merge with mnist1525444683.1159678: 0.9435
Server performance after merge with mnist1525444737.431835: 0.9453
Server performance after merge with mnist1525444790.946187: 0.9466
Server performance after merge with mnist1525444843.6395044: 0.9442
Server performance after merge with mnist1525444898.636411: 0.9415
Server performance after merge with mnist1525446266.6451032: 0.9445
Server performance after merge with mnist1525446321.1786842: 0.9447
Server performance after merge with mnist1525446375.3714283: 0.9464
Server performance after merge with mnist1525446433.1926417: 0.9435
Server performance after merge with mnist1525446522.1355023: 0.9429
Server performance after merge with mnist1525446576.4898996: 0.9449
Server performance after merge with mnist1525446629.